- 1) Calculate the dot product between two word embeddings which you believe are similar

In [22]:
import numpy as np
import torch as nn
import gensim.downloader as api

In [ ]:
model = api.load("glove-wiki-gigaword-50")

In [11]:
#Two wordembeddings
apple_vec = model["apple"]
banana_vec = model["banana"]

#dot product (=length of the projected vector)
np.dot(apple_vec, banana_vec)

14.41781

- 2) Calculate the dot product between the two word and a word which you believe is dissimilar

In [13]:
war_vec = model["war"]

#dot
print(np.dot(apple_vec, war_vec)) #lower
print(np.dot(banana_vec, war_vec)) #lower

5.6469955
6.790193



- 3) make the three words into a matrix $E$ and multiply it with its own transpose using matrix multiplication. So $E \cdot E^T$
  - what does the values in matric correspond to? What do you imagine the dot product is? *Hint*, similarity between vectors (cosine similarity) is exactly the same as the dot product assuming you normalize the lenghth of each vector to 1.

In [21]:
E = np.asmatrix([apple_vec, banana_vec, war_vec])
print(E.shape)

#transpose
E_t = E.T

#matrix multipication @
v = E @ E.T
print(v)

# Matrix values in the diagonal (top/left to bottom/right) is the dot product between itself and itself -
# that's why they are so similar.
# corresponds to the dot product between all combinations

(3, 50)
[[28.009117 14.417807  5.646996]
 [14.417807 23.599033  6.790193]
 [ 5.646996  6.790193 35.105724]]



- 4) Examine the attention formula from Vaswani et al. (2017), you have now implemented $Q\cdot K^T$
$$
Attention(Q, K, V) = softmax(\frac{Q\cdot K^T}{\sqrt{d}}) \cdot V
$$
Where $d$ is the dimension of of the embedding and Q, K, V stands for queries, keys and values.


  - 4.1) Now add the softmax. Examining the outcome, how come that the matrix is no longer symmetric?


In [29]:
#adding softmax
v_ten = nn.tensor(v)
soft = nn.softmax(v_ten, dim = 1)
print(soft)

#back to np
soft = np.asarray(soft)
print(soft)

#not symmetric (cannot be mirrored in the diagonal)
print(sum(soft[0]))
print(sum(soft[1]))
print(sum(soft[2]))
#all rows add up to 1

tensor([[1.0000e+00, 1.2513e-06, 1.9420e-10],
        [1.0294e-04, 9.9990e-01, 5.0115e-08],
        [1.6078e-13, 5.0434e-13, 1.0000e+00]])
[[9.9999881e-01 1.2513199e-06 1.9420189e-10]
 [1.0294356e-04 9.9989700e-01 5.0115133e-08]
 [1.6078171e-13 5.0433746e-13 1.0000000e+00]]
1.000000059421179
0.9999999968478477
1.000000000000665


  - 4.2) Now normalize the using the $\sqrt{d}$, how does this change the outcome?

1) the matrix resulting from the softmax is referred to as the attention matrix and is how much each matrix should pay attention to the others when we multiply our attention matrix by our matrix $E$ (corresponding to $V$). Try it out:

- 5.1) This is essentially a weighted sum, one way to see this is to extract the weight from the first row of the matrix